<div style="color:red;font-size:40px;">
<center>
<font size="6" >
به نام خدا 
<br><br>
<font size="5" color="grey" >
<b>
Github
سایت
README
بازیابی بر روی فایل‌های 

</b>
<br>
<font size="4" color="black">
دانشجویان:
<br>
 (98101339) سایه جارالهی 
<br>
 (98171104) بردیا محمدی 
<br>
امیررضا سلیمان‌بیگی (98109837)
<br>
استاد: دکتر احسان‌الدین عسگری
</center>
<br>
</div>

## Crawling

First, the url of different repos are crawled. Consequently, for each repo, th README.md is crawled. Also, empty readmes are ignored.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
import re 

def get_readme(url):
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    html_code = soup.find("article")
    sents = []
    for p in html_code.find_all(['td','p', 'li', re.compile('^h\d+$')]):
        text = p.get_text()
        if text and text.replace('\n', ''):
            sents.append(text)
    return sents


def scrape_readmes(url_list):
    readmes = []
    for idx, url in enumerate(url_list):
        print(f"{idx + 1} / {len(url_list)} | Accessing {url}")
        try:
            readme = get_readme(url)
            if len(readme):
                readmes.append({"url": url, "readme_html": readme})
        except:
            print(f"Couldn't access {url}")
        time.sleep(3)  # needed to not overload GitHub -> getting blocked
    readmes_df = pd.DataFrame(readmes)
    return readmes_df

In [ ]:
topics = [
    'game',
    'hardware', 
    'computer+vision', 
    'artificial+intelligence', 
    'application', 
    'ios', 
    'multiplatform', 
    'deep+learning', 
    'cloud', 
    'verilog'
]
valid_topic_urls = [f'https://github.com/search?o=desc&p={i}&q={topic}&s=stars&type=Repositories' for i in range(1, 21) for topic in topics]


In [ ]:
def get_repo_urls(url):
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    repos = soup.find_all('a', class_='v-align-middle')
    time.sleep(3) # needed to not overload GitHub -> getting blocked
    return ['https://github.com'+repo.get('href') for repo in repos]

def scrape_urls():
    repo_urls = []
    for i, url in enumerate(valid_topic_urls):
        print('scraping url', i)
        if url not in repo_urls:
            repos = get_repo_urls(url)
            repo_urls.extend(repos)
    return repo_urls
    

In [ ]:
repo_urls = scrape_urls()

In [ ]:
f = open('urls.json')
repo_urls = json.load(f)
readmes_df = scrape_readmes(repo_urls)
readmes_df.to_json("readmes.json", orient="records")

##Load data from files

In [37]:
readmes_df = pd.read_json('readmesjson.json', orient= 'records')
tokens_df = pd.read_json('token.json', orient= 'records')
paragraph_df = pd.read_json('string.json', orient='records')

## Preprocess(Normalization, Tokenizarion, Removing Stopwords)

In [9]:
import nltk
import string
import json
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
from pathlib import Path

class Preprocess:
    
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer  = PorterStemmer()

    def remove_emoji(self, string):
        b'\\U0001f923'
        emoj = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642" 
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
                        "]+", re.UNICODE)
        return emoj.sub(r'', string)

    def normalize_tokens(self, tokens, remove_stopword, stopwords_domain=['https']):
        
        # Remove stopwords
        if remove_stopword:
            stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
            normalized_tokens = [word for word in tokens if (word.lower() not in stopwords_domain + stopwords)]
        else:
            normalized_tokens = tokens
            
        # remove links
        normalized_tokens = [word for word in normalized_tokens if not re.match(r'^//.*(/?)$', word)]
        
        # Remove punctuations
        normalized_tokens = [word for word in normalized_tokens if word not in string.punctuation]
        
        # Convert everything to lowercase
        normalized_tokens = [word.lower() for word in normalized_tokens]

        normalized_tokens = [word for word in normalized_tokens if len(word)>1]
        
        
        # Convert numbers to "Number" token
        normalized_tokens = [word if not re.match(r'(^| )\d+(\.\d+)*($| )', word) else 'NUMBER' for word in normalized_tokens]
            
        return normalized_tokens

    def isEnglish(self, s):
        try:
            s.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        else:
            return True
        
        
    def non_english(self, text):
        t_list = word_tokenize(text)
        res = []
        for i in t_list:
            if self.isEnglish(i):
                res.append(i)
        return ' '.join(res)



    def preprocess(self, text, lemmatize=True, remove_stopword=True, token=True, re_sub_domain='\'|`|-'):

        # remove non enlish
        text = self.non_english(text)

        # replace sub_domain with space
        text = re.sub(re_sub_domain, ' ', text)

        # remove emoji
        text = self.remove_emoji(text)

        tokens = word_tokenize(text)
        
        # normalization
        tokens = self.normalize_tokens(tokens, remove_stopword)
        
        # lemmatization
        if lemmatize:
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        
        if token:
            return tokens
        else:
            return ' '.join(tokens)


    def get_preprocessed_token(self, file_path='readmes.json'):
        data = json.loads(Path(file_path).read_text())
        docs = []
        for row in data:
            text = row['readme_html']
            preprocessed_tokens = []
            for paragraph in text:
                preprocessed_tokens += self.preprocess(paragraph)
            docs.append({'url': row['url'], 'text': preprocessed_tokens})
        Path('token.json').write_text(json.dumps(docs))


    def get_preprocessed_str(self, file_path='readmes.json'):
        data = json.loads(Path(file_path).read_text())
        docs = []
        for row in data:
            text = row['readme_html']
            preprocessed_paragraphs = []
            for paragraph in text:
                preprocessed_paragraphs.append(self.preprocess(paragraph, lemmatize=False, remove_stopword=False, token=False))
            docs.append({'url': row['url'], 'text': preprocessed_paragraphs})
        Path('string.json').write_text(json.dumps(docs))

In [ ]:
Preprocess().get_preprocessed_token()

In [ ]:
Preprocess().get_preprocessed_str()

## Boolean Retrival
In this section, boolean retrival is implemented. The format of input query should be: 

word1 AND word2 word3 OR word4 word5 NOT word6

The matrix is implemented using inverted index. 


In [39]:
import json 
import random
from copy import deepcopy
class BooleanRetrival:

    def __init__(self, docs, k = 10):
        self.inverted_index = {}
        self.docs = docs
        self.query_keywords = ['NOT', 'ORNOT', 'OR', 'AND']
        self.k = k 
        self.create_inverted_index()
    
    def create_inverted_index(self):
        for i, row in self.docs.iterrows():
            tokens = row['text']
            url = row['url']
            for word in tokens:
                if word in self.inverted_index.keys():
                    self.inverted_index[word].add(url)
                else:
                    self.inverted_index.update({word:{url}})
    
    def dump_dict_json(self):
        with open('inverted_index.json', 'w') as f:
            f.write(json.dumps(self.inverted_index))

    def add_and_to_query(self, query):
        temp_query = deepcopy(query)
        changed_count = 0 
        for i in range(len(query)-1):
            if query[i] not in self.query_keywords and query[i+1] not in self.query_keywords:
                temp_query.insert(i+1 + changed_count,'AND')
                changed_count +=1 
        return temp_query
    
    def normalize_query(self, query):
        result = []
        for text in query:
            if text not in self.query_keywords:
                result.append(Preprocess().preprocess(text, token = False))
            else:
                result.append(text)
        return result
        
    def tokenize_query(self, query):
        query = query.replace('OR NOT', 'ORNOT')
        query = query.split(' ')
        query = self.add_and_to_query(query)
        query = self.normalize_query(query)
        return query
    
    def process_query(self, query, query_expansion = False):
        if not query_expansion:
            query = self.tokenize_query(query)
        if not query[0] in self.inverted_index.keys():
            result = set(self.docs.loc[:,"url"])
        else:
            result = set(self.inverted_index[query[0]])
        i = 1
        while i < len(query):
            result = self.handle_operation(result,query[i+1], query[i])
            i += 2
        res = []
        for i in result:
            res.append({'score':1.0, 'url': i})
        if query_expansion:
            return result
        return res
    
    def handle_operation(self, result:set, second_word:str, operation:str):
        if not second_word in self.inverted_index.keys():
            return result
        second_list = self.inverted_index[second_word]
        if operation == 'AND':
            return result.intersection(second_list)
        elif operation=='OR':
            return result.union(second_list)
        elif operation == 'ORNOT' :
            all_urls = self.docs.loc[:,"url"]
            return result.union((set(tokens_df.loc[:,'url'])) - self.inverted_index[second_word])
        elif operation == 'NOT':
            return result - second_list

    def expand_query(self, query):
        ft_model = FastText.load('_fasttext.model')
        query = self.tokenize_query(query)
        all_queries = [query]
        for i in range(5):
            indexes = {}
            for index, part in enumerate(query):
                if part in ['ORNOT', 'NOT', 'OR', 'AND']:
                    continue
                similars = [word for word, percentage in ft_model.most_similar([part]) if percentage>0.85]
                indexes.update({index: similars})
            temp, query = self.generate_queries(indexes, query)
            all_queries.extend(temp)
            all_queries.append(query)
        res = set()
        for query in all_queries:
            res.update(self.process_query(query, True))
        return res
            


    def generate_queries(self, indexes, query):
        final_query = deepcopy(query)
        new_queries = []
        for index, similars in indexes.items():
            if similars:
                final_query[index] = random.choice(similars)
                q = deepcopy(query)
                q[index] = random.choice(similars)
                new_queries.append(q)
        return new_queries, final_query





In [40]:
from preprocess import tokens_df
model = BooleanRetrival(tokens_df)


In [41]:
result = model.process_query('Hardware cores OR Arduino AND API')
result

[{'score': 1.0, 'url': 'https://github.com/microsoft/DirectML'},
 {'score': 1.0, 'url': 'https://github.com/BradLarson/GPUImage'},
 {'score': 1.0, 'url': 'https://github.com/chipsalliance/Surelog'},
 {'score': 1.0, 'url': 'https://github.com/NervanaSystems/neon'},
 {'score': 1.0, 'url': 'https://github.com/oshi/oshi'},
 {'score': 1.0,
  'url': 'https://github.com/ahkarami/Deep-Learning-in-Production'},
 {'score': 1.0, 'url': 'https://github.com/cocotb/cocotb'},
 {'score': 1.0, 'url': 'https://github.com/PaddlePaddle/Paddle'},
 {'score': 1.0,
  'url': 'https://github.com/robmarkcole/satellite-image-deep-learning'},
 {'score': 1.0, 'url': 'https://github.com/pikvm/pikvm'},
 {'score': 1.0, 'url': 'https://github.com/matteocrippa/awesome-swift'},
 {'score': 1.0, 'url': 'https://github.com/nolimits4web/swiper'},
 {'score': 1.0, 'url': 'https://github.com/ellisonleao/magictools'},
 {'score': 1.0,
  'url': 'https://github.com/awesome-selfhosted/awesome-selfhosted'},
 {'score': 1.0, 'url': 'ht

In [35]:
len(result)

43

In [43]:
result = model.expand_query('Hardware cores OR Arduino AND API')
result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


44

## Shunt algorithm: 
This can be used if our queries have paranthesis or they are inside each other. As was mentioned in quera, there is no need for the boolean retrival module to handle queries with paranthesis. However, we implemented that feature. 

In [ ]:
from collections import namedtuple

opinfo = namedtuple('Operator', 'precedence associativity')
operator_info = {
    "AND": opinfo(0, "L"),
    "OR": opinfo(0, "L"),
    "NOT": opinfo(0, "L"),
    "ORNOT": opinfo(0,"L")
}

def shunt(tokens):
    tokens += ['end']
    operators = []
    output = []
    while len(tokens) != 1:
        current_token = tokens.pop(0)

        if current_token not in operator_info.keys() and current_token not in ['(', ')']:
            output.append(current_token)
        elif current_token in operator_info.keys():
            while len(operators):
                satisfied = False
                if operators[-1] not in ["(", ")"]:
                    satisfied = True
                satisfied = satisfied and operators[-1] != "("
                if not satisfied:
                    break
                output.append(operators.pop())
            operators.append(current_token)
        elif current_token == "(":
            operators.append(current_token)

        elif current_token == ")":
            while len(operators) and operators[-1] != '(':
                output.append(operators.pop())
            if len(operators) != 0 and operators[-1] == "(":
                operators.pop()
    output.extend(operators[::-1])
    return output



## Tf-idf

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
from pathlib import Path

class Tfidf:
    
    def __init__(self, file_name='token.json', k = 10):
        data = json.loads(Path(file_name).read_text())
        corpus = []
        self.urls = []
        for row in data:
            corpus.append(' '.join(row['text']))
            self.urls.append(row['url'])

        self.vectorizer = TfidfVectorizer(
            input='content', 
            lowercase=False, 
            analyzer='word', 
            ngram_range=(1, 1), 
            max_df=0.90, 
            min_df=1, 
            max_features=None, 
            vocabulary=None, 
            binary=False, 
            norm='l2', 
            use_idf=True, 
            smooth_idf=True, 
            sublinear_tf=True
        )
        self.k = k
        self.X = self.vectorizer.fit_transform(corpus)
        self.words = self.vectorizer.get_feature_names_out().tolist()
        
    def process_query(self, query, url_num=10, expand = False):
        query = ' '.join(Preprocess().preprocess(query))
        query_embedding = self.vectorizer.transform([query]).T
        if expand:
            return self.expand_query(query_embedding.T)     
        score = self.X.dot(query_embedding).toarray()[:, 0]
        max_scores_docs = list(np.argsort(score))
        
        related_urls = []
        scores = []
        for doc in reversed(max_scores_docs):
            if self.urls[doc] not in related_urls:
                related_urls.append(self.urls[doc])
                scores.append(score[doc])
            if len(related_urls) == url_num:
                break
            
        return related_urls, scores
        

    def print_results(self, query, expand = False):
        for url, score in zip(*self.process_query(query, expand =  expand)):
            print(f'url: {url}, score: {score:.5f}')
        
    def expand_query(self,query):
        _, url, cos_sim = Rocchio().expand_query(self.X.toarray(), query.toarray()[0], self.urls)
        return url[0][0:self.k], cos_sim[0][0:self.k]

In [115]:
model = Tfidf()


In [102]:
model.print_results(query='face detection')

url: https://github.com/cvzone/cvzone, score: 0.28570
url: https://github.com/arunponnusamy/cvlib, score: 0.15126
url: https://github.com/php-opencv/php-opencv-examples, score: 0.14787
url: https://github.com/chandrikadeb7/Face-Mask-Detection, score: 0.14187
url: https://github.com/eduardolundgren/tracking.js, score: 0.14162
url: https://github.com/mpatacchiola/deepgaze, score: 0.13612
url: https://github.com/bnosac/image, score: 0.13437
url: https://github.com/symisc/sod, score: 0.13375
url: https://github.com/extreme-assistant/survey-computer-vision-2020, score: 0.12323
url: https://github.com/amusi/awesome-ai-awesomeness, score: 0.11560


In [116]:
model.print_results(query = 'face detection', expand = True)

(1475, 59504) (1, 59504) <class 'list'>
<class 'numpy.ndarray'>
url: https://github.com/cvzone/cvzone, score: 0.46401
url: https://github.com/arunponnusamy/cvlib, score: 0.35401
url: https://github.com/mpatacchiola/deepgaze, score: 0.35140
url: https://github.com/php-opencv/php-opencv-examples, score: 0.33587
url: https://github.com/chandrikadeb7/Face-Mask-Detection, score: 0.32436
url: https://github.com/bnosac/image, score: 0.32299
url: https://github.com/eduardolundgren/tracking.js, score: 0.32273
url: https://github.com/symisc/sod, score: 0.20424
url: https://github.com/rchavezj/OpenCV_Projects, score: 0.19810
url: https://github.com/amzn/computer-vision-basics-in-microsoft-excel, score: 0.18396


## FastText

In [ ]:
! pip install engines


In [65]:
from gensim.models.fasttext import FastText
import numpy as np
import pandas as pd
import scipy

class FastTextModel:

    def __init__(self, data, train=False, k=-1):
        self.data = data
        if train:
            self.ft_model = self.train()
        else:
            self.ft_model = FastText.load('_fasttext.model')
        self.data_embedding = self.get_data_embedding_avg()
        self.k = k

    def get_words(self, doc):
        words = []
        for sent in doc:
            words.extend(sent.split(' '))
        return words

    def get_tok_text(self):
        tok_text = []
        for doc in self.data['readme_html']:
            tok_text.append(self.get_words(doc))
        return tok_text

    def train(self):
        ft_model = FastText(sg=1, size=100, window=10, min_count=2, negative=15, min_n=2, max_n=5)
        tok_text = list(self.data.loc[:, 'text'])
        ft_model.build_vocab(tok_text)
        ft_model.train(
            tok_text,
            epochs=6,
            total_examples=ft_model.corpus_count,
            total_words=ft_model.corpus_total_words)
        ft_model.save('_fasttext.model')  # save
        return ft_model

    def get_data_embedding_avg(self) -> dict:
        '''

        :return: a dict: the keys are the urls of data and the value is the embedding average
        '''
        docs_avg = dict()
        for index, row in self.data.iterrows():
            words = row['text']
            url = row['url']
            docs_avg[url] = np.mean([self.ft_model.wv[word] for word in words], axis=0)
        return docs_avg

    def get_query_embedding(self, query_tokens):
        return np.mean([self.ft_model.wv[word] for word in query_tokens], axis=0)

    def tokenize_query(self, query: str):
        query = Preprocess().preprocess(query, token=True)
        return query

    def search_query(self, query, expand = False):
        tokens = self.tokenize_query(query)
        q_embedding = self.get_query_embedding(tokens)
        if expand:
            return self.expand_query(q_embedding)
        cosine_sim_values = {}
        for url, embedding_value in self.data_embedding.items():
            cosine_sim_values.update({url: self.cosine_sim(embedding_value, q_embedding)})
        if not expand:
            return self.calculate_best_k(cosine_sim_values)


    def calculate_best_k(self, cosine_sim_values:dict):
        res = sorted(cosine_sim_values.items(), key=lambda x: x[1], reverse = True) [0: self.k]
        return res

    def cosine_sim(self, x, y):
        return 1- scipy.spatial.distance.cosine(x, y)

    def expand_query(self,q_embedding ):
        _, url, cos_sim = Rocchio().expand_query([x for u, x in self.data_embedding.items()],q_embedding, [u for u, x in self.data_embedding.items()])
        res = dict()
        for i in range(self.k):
            res.update({url[0][i]:cos_sim[0][i]})
        return res.items()

## Initialize FastText model 

In [66]:
# by setting train value to True, you have to wait about 8 minutes for the training process
ft_model = FastTextModel(tokens_df, train=False, k=10)
a = ft_model.search_query('atari game')

In [67]:
a = ft_model.search_query('internet of things circuit')
a

[('https://github.com/itsFrank/MinecraftHDL', 0.8557753562927246),
 ('https://github.com/dawsonjon/Chips-2.0', 0.8528292775154114),
 ('https://github.com/yadox666/The-Hackers-Hardware-Toolkit',
  0.8504918813705444),
 ('https://github.com/kitspace/awesome-electronics', 0.8503836989402771),
 ('https://github.com/fossasia/neurolab-hardware', 0.8484646081924438),
 ('https://github.com/aappleby/MetroBoy', 0.8444978594779968),
 ('https://github.com/fossasia/neurolab-firmware', 0.8440712690353394),
 ('https://github.com/pervognsen/bitwise', 0.8423163890838623),
 ('https://github.com/OpnTec/open-spectrometer-hardware', 0.8421347737312317),
 ('https://github.com/stevehoover/warp-v', 0.8387736678123474)]

In [68]:
a = ft_model.search_query('internet of things circuit', expand = True)
list(a)

<class 'numpy.ndarray'>


[('https://github.com/OpnTec/open-spectrometer-hardware', 0.9373492),
 ('https://github.com/yadox666/The-Hackers-Hardware-Toolkit', 0.9358752),
 ('https://github.com/kitspace/awesome-electronics', 0.93562627),
 ('https://github.com/dawsonjon/Chips-2.0', 0.93493414),
 ('https://github.com/fossasia/neurolab-hardware', 0.93459404),
 ('https://github.com/itsFrank/MinecraftHDL', 0.93376243),
 ('https://github.com/pervognsen/bitwise', 0.933619),
 ('https://github.com/fossasia/neurolab-firmware', 0.93012476),
 ('https://github.com/m-labs/migen', 0.9258059),
 ('https://github.com/stevehoover/warp-v', 0.92271626)]

## Initialize BERT model 

We will use the distilbert-base-nli-stsb-mean-tokens model.

Faiss is a library for efficient similarity search and clustering of dense vectors.


In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers


In [61]:


import numpy as np
import pandas as pd
import faiss

import torch
from sentence_transformers import SentenceTransformer
import scipy 
from operator import itemgetter

class TransformerModel:

    def __init__(self, data, k = 10, load_data = True):
        self.k = 10
        self.model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
        if torch.cuda.is_available():
            model = self.model.to(torch.device("cuda"))
        self.data = data
        self.list_to_string()
        self.set_id()
        if load_data:
            self.embedding = np.load('bert_embedding.npy')
        else:
            self.embedding = self.get_data_embedding()
        self.index = self.set_indexes()

    def list_to_string(self):
      #Change list of doc string to string
        for i in range(0, len(self.data['readme_html'])):
          self.data['readme_html'][i] = ' '.join(map(str, self.data['readme_html'][i]))

    def get_data_embedding(self):
        return self.model.encode(self.data['readme_html'].tolist(), show_progress_bar=True)

    def vector_search(self, query, model, index, num_results=10):
        vector = model.encode(list(query))
        D, I = index.search(np.array(vector).astype("float32"), k=num_results)
        return D, I

    def set_id(self):
        self.data['id'] = range(0, len(self.data))
    
    def set_indexes(self):
        embeddings = np.array([embedding for embedding in self.embedding]).astype("float32")
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index = faiss.IndexIDMap(index)
        index.add_with_ids(embeddings, self.data.id.values)
        return index

    def get_recommendation(self, user_query):
        D, I = self.vector_search([user_query], self.model, self.index, num_results=10)
        print(f'L2 distance: {D.flatten().tolist()}\n\nReadme IDs: {I.flatten().tolist()}')
        return [self.data['url'][i] for i in I.flatten().tolist()]

    def get_recommendation2(self, query, expand = False):
        queries = [query]
        query_embeddings = self.model.encode(queries)
        if expand:
            return self.expand_query(query_embeddings[0])
        number_top_matches = 10
        for query, query_embedding in zip(queries, query_embeddings):
            distances = scipy.spatial.distance.cdist(query_embeddings, self.embedding, "cosine")[0]
            results = zip(range(len(distances)), distances)
            results = sorted(results, key=lambda x: x[1])
            temp = []
            for idx, distance in results[0:number_top_matches]:
              temp.append(str(1-distance) + ' ' + readmes_df['url'][idx])
            return temp

    def expand_query(self,q_embedding ):
        _, url, cos_sim = Rocchio().expand_query(self.embedding,q_embedding, np.array(readmes_df['url']))
        print('212')
        temp = []
        for i in range(self.k):
            temp.append(str(cos_sim[0][i]) + ' '+ url[0][i])
        return temp


In [62]:
t_model = TransformerModel(readmes_df)
a = t_model.get_recommendation2('password manager')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
a

['0.6634643535739695 https://github.com/marcwebbie/passpie',
 '0.6069893094109988 https://github.com/keepassxreboot/keepassxc',
 '0.457747353179063 https://github.com/nndl/nndl.github.io',
 '0.4214187736338315 https://github.com/nondanee/UnblockNeteaseMusic',
 '0.41270995062754745 https://github.com/blinker-iot/blinker-js',
 '0.41015671358868366 https://github.com/jpillora/cloud-torrent',
 '0.4077809766286429 https://github.com/relativty/Relativty',
 '0.4050003055261049 https://github.com/pikvm/pikvm',
 '0.4022530244289647 https://github.com/ndeadly/MissionControl',
 '0.3958997887211033 https://github.com/FiloSottile/mkcert']

In [63]:
a = t_model.get_recommendation2('password manager', True)
a

<class 'numpy.ndarray'>
212


['0.8459593 https://github.com/marcwebbie/passpie',
 '0.7843307 https://github.com/keepassxreboot/keepassxc',
 '0.678036 https://github.com/nondanee/UnblockNeteaseMusic',
 '0.6655154 https://github.com/jpillora/cloud-torrent',
 '0.6591674 https://github.com/Vove7/yyets_flutter',
 '0.63441026 https://github.com/blinker-iot/blinker-js',
 '0.62113225 https://github.com/ish-app/ish',
 '0.61276305 https://github.com/ArcadeRenegade/SidebarDiagnostics',
 '0.6116522 https://github.com/relativty/Relativty',
 '0.60775113 https://github.com/canonical/multipass']

## Report

<div dir="rtl">
<font size=5>
<b>
گزارش انجام پروژه:
</b>
<font size =3>
<br>
در این تمرین، فایل‌های README گیت‌هاب با استفاده از crawling جمع آوری شده و پس از انجام پیش پردازش بر روی آن‎‌ها، چهار مدل مختلف برای بازیابی اطلاعات و جستجوی مطالب قرار گرفته است. 
<br>
<b>Crawling:</b><br>
در گام اول Crawling داده انجام شده است. با توجه به اینکه در گیت‌هاب ریپازیتوری های مختلفی با موضوعات بسیار گوناگون وجود دارد، با جمع آوری داده در اندازه محدود‌ (حدود ۱۶۰۰ ریپازیتوری)‌ نمیتوان به طور مناسب جستجو را انجام داد. به این جهت، کرال کردن داده ها بر روی ۱۰ دسته کلی از ریپازیتوری‌ها (بازی، سخت افزار، بینایی ماشین، هوش مصنوعی، اپلیکیشن، ios، مولتی پلتفرم، دیپ لرنینگ، فضای ابری، وریلاگ) انجام شده است. از هریک از این ریپازیتوری ها 20 صفحه ( که هر یک شامل 10 ریپازیتوری است) ذخیره شده و README آنها مورد بررسی قرار گرفته است. 
<br>
<b>PreProcessing:</b>
<br>
چالش بعدی در این زمینه، Preprocessing بر روی داده ها برای رسیدن به داده مطلوب بوده است. در این بخش، در صورتی که داده ها به زبانی به جز انگلیسی باشند (موردی که مشاهده شد آن است که داده هایی به زبان چینی و هندی بسیار رایج است) آن داده ها را حذف میکند. همچنین از lemmatizer استفاده میشود تا کلمات همخانواده به یک کلمه یکسان بروند. در نهایت نیز از تمام اعداد با عبارت NUMBER جانشین شده، از Tokenizerو Normalizer و Stopword Removal استفاده شده است تا خروجی مناسب تولید شود. در برخی از الگوریتم های بازیابی از توکن ها و در برخی از متن کامل نورمالایزر شده استفاده میشود. همچنین در بسیاری از repository ها لینک‌هایی وجود دارد که آنها را نیز حذف می‌کنیم. در صورتی طول کلمه کمتر از 2 باشد آن را حذف می‌کنیم. همچنین emoji های موجود را نیز حذف می‌کنیم.
<br>

</div>


<div dir= "rtl">
<font size =3>
<b>Boolean Retrival:</b>
<br>
در گام اول جستجو Boolean انجام می‌شود. این جستجو تنها به حضور کلمات در متن‌ها توجه دارد. در نتیجه هرچه تعداد کلمات کوئری‌ها بیشتر باشد و این کلمات به هم نامرتبط باشد،‌ تعداد خروجی‌ها نیز کمتر می‌شود. برای استفاده از این الگوریتم، ابتدا inverted matrix از روی داده‌ها به دست آورده میشود و در بخش بعدی، پاسخ دادن به کوئری‌ها انجام می‌شود. فرمت کوئری‌ها به صورتی است که تعدادی کلمه است که با AND, OR, NOT, OR NOT جدا شده اند. درصورتی‌که بین دو کلمه هیچ عبارت متصل کننده‌ای نباشد نیز AND در نظر گرفته میشود. 
مقدار mrr عدد 0.61 است

</div>

In [117]:
result = model.process_query('atari game NOT deep')
result

(['https://github.com/keras-rl/keras-rl',
  'https://github.com/OpenEmu/OpenEmu',
  'https://github.com/yenchenlin/DeepLearningFlappyBird',
  'https://github.com/amusi/Deep-Learning-Interview-Book',
  'https://github.com/ardamavi/Game-Bot',
  'https://github.com/watabou/pixel-dungeon',
  'https://github.com/KestrelComputer/kestrel',
  'https://github.com/simoninithomas/Deep_reinforcement_learning_Course',
  'https://github.com/charcole/Z3',
  'https://github.com/andri27-ts/Reinforcement-Learning'],
 [0.2043208221190701,
  0.17373314306640214,
  0.14216671035472217,
  0.12498932585085298,
  0.10777785843395163,
  0.10263801713559492,
  0.0996560499631102,
  0.09510212303684056,
  0.0913781407004026,
  0.0900197024776015])

In [118]:
result = model.process_query('cloud service OR IoT AND docker')
result

(['https://github.com/thingsboard/thingsboard',
  'https://github.com/wxiaoqi/Spring-Cloud-Platform',
  'https://github.com/macrozheng/springcloud-learning',
  'https://github.com/dyc87112/SpringCloud-Learning',
  'https://github.com/localstack/localstack',
  'https://github.com/docker/compose',
  'https://github.com/netdata/netdata',
  'https://github.com/paascloud/paascloud-master',
  'https://github.com/blinker-iot/blinker-library',
  'https://github.com/ityouknow/spring-cloud-examples'],
 [0.22630859780378812,
  0.189461093555495,
  0.1836757297692159,
  0.16341134293341783,
  0.15013899194256636,
  0.1421791969205611,
  0.12978398410466505,
  0.12674046466582461,
  0.12633112249324804,
  0.1190655953187189])

In [119]:
result = model.process_query('hybrid transactional processing OR recommender OR system')
result

(['https://github.com/pingcap/tidb',
  'https://github.com/d2l-ai/d2l-en',
  'https://github.com/shenweichen/DeepCTR',
  'https://github.com/cryptomator/cryptomator',
  'https://github.com/LumingSun/ML4DB-paper-list',
  'https://github.com/halfrost/Halfrost-Field',
  'https://github.com/mdipietro09/DataScience_ArtificialIntelligence_Utils',
  'https://github.com/cockroachdb/cockroach',
  'https://github.com/fnproject/fn',
  'https://github.com/Tencent/VasSonic'],
 [0.10713447896343649,
  0.06813319305446196,
  0.06122971289014981,
  0.057355690441885765,
  0.05331318891452002,
  0.05201791861750958,
  0.048158459087971194,
  0.047936336649312856,
  0.046609899910211416,
  0.04508615318328211])

In [120]:
result = model.process_query('Kubernetes OR docker AND tutorial')
result

(['https://github.com/BretFisher/udemy-docker-mastery',
  'https://github.com/kelseyhightower/kubernetes-the-hard-way',
  'https://github.com/googleforgames/agones',
  'https://github.com/apache/openwhisk',
  'https://github.com/ToolJet/ToolJet',
  'https://github.com/armosec/kubescape',
  'https://github.com/dgkanatsios/CKAD-exercises',
  'https://github.com/docker/compose',
  'https://github.com/falcosecurity/falco',
  'https://github.com/dotnet-architecture/eShopOnContainers'],
 [0.26197910474737485,
  0.2389234022954762,
  0.2265280444282356,
  0.19445904048361248,
  0.16016655444814767,
  0.14618747608648072,
  0.14523016885373158,
  0.14011337790951978,
  0.13744752510717267,
  0.13192309827125104])

In [121]:
result = model.process_query('accelerate deep learning applications AND GPU OR CPU')
result

(['https://github.com/amusi/Deep-Learning-Interview-Book',
  'https://github.com/NVIDIA/DALI',
  'https://github.com/thibo73800/aihub',
  'https://github.com/NervanaSystems/neon',
  'https://github.com/tflearn/tflearn',
  'https://github.com/symisc/sod',
  'https://github.com/microsoft/computervision-recipes',
  'https://github.com/pierpaolo28/Artificial-Intelligence-Projects',
  'https://github.com/eclipse/deeplearning4j',
  'https://github.com/floydhub/dl-docker'],
 [0.17176341255023447,
  0.17116339327684793,
  0.1608545304953879,
  0.13656153990823058,
  0.1307579381027511,
  0.1281080389708672,
  0.12735904654914149,
  0.12479473138052749,
  0.12072464613463144,
  0.11951162854857461])

In [122]:
result = model.process_query('React web application development NOT webStorm')
result


(['https://github.com/react-native-elements/react-native-elements',
  'https://github.com/facebook/react-native',
  'https://github.com/invertase/react-native-firebase',
  'https://github.com/microsoft/fluentui',
  'https://github.com/alan2207/bulletproof-react',
  'https://github.com/reactide/reactide',
  'https://github.com/GeekyAnts/vue-native-core',
  'https://github.com/feathersjs/feathers',
  'https://github.com/midwayjs/midway',
  'https://github.com/roughike/inKino'],
 [0.15496579495813756,
  0.14177082182649722,
  0.13259730273182554,
  0.1310721414397063,
  0.12409283716008174,
  0.12186330485250957,
  0.12117623524967125,
  0.12050926755228072,
  0.11442715371979514,
  0.11341564505209982])

<div dir = "rtl">
<font size=3>
<b>Shunt Algorithm for Boolean Retrival:</b> 
</div>

<div dir="rtl">
الگوریتم Shunt یک الگوریتم برای تبدیل یک عبارت infix به postfix است. درصورتی که کوئری وارد شده دارای اولویت و پرانتزگذاری باشد، ابتدا این الگوریتم را روی کوئری صدا میزنیم. پس از آنکه کوئری به حالت postfix در بیاید میتوان با اجرای کوئری و استفاده از استک برای نگه داری بخش های مختلف کوئری،‌ جواب را پیدا کرد. 
</div>

<div dir = "rtl">
<font size=3>
<b>Tf-idf:</b> 
</div>

<div dir="rtl">
از کلاس TfidfVectorizer در ماژول sklearn.feature_extraction.text استفاده می‌کنیم. 'input='content را استفاده می‌کنیم که ورودی را string در نظر بگیرد، 'analyzer='word استفاده می‌کنیم تا پردازش در سطح کلمه باشد. از مدل unigram نیز استفاده می‌کنیم. کلماتی که در بیشتر از 0.9 داکیومنت‌ها هستند را نادیده می‌گیریم. از 'L2 norm' برای normalize هر سطر استفاده می‌کنیم. همچنین به جای tf از 1 + log(tf) برای به جای tf در رابطه tf-idf استفاده می‌کنیم. از smoothing نیز استفاده می کنیم به طوری که یک را با فرکانس داکیومنت‌ها جمع می‌کنیم تا از تقسیم شدن به صفر جلوگیری کند. MRR در این بخش برابر با 0.82 است.
</div>

In [123]:
model = Tfidf()
model.print_results(query='applications of deep learning')

url: https://github.com/amusi/Deep-Learning-Interview-Book, score: 0.33808
url: https://github.com/thibo73800/aihub, score: 0.31660
url: https://github.com/ankonzoid/artificio, score: 0.23456
url: https://github.com/tirthajyoti/Papers-Literature-ML-DL-RL-AI, score: 0.20274
url: https://github.com/ashishpatel26/500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code, score: 0.19498
url: https://github.com/rasmusbergpalm/DeepLearnToolbox, score: 0.19268
url: https://github.com/openai/spinningup, score: 0.18900
url: https://github.com/fchollet/deep-learning-with-python-notebooks, score: 0.18619
url: https://github.com/PacktPublishing/PyTorch-Computer-Vision-Cookbook, score: 0.18137
url: https://github.com/khanhnamle1994/computer-vision, score: 0.18107


In [124]:
model = Tfidf()
model.print_results(query='Game studio projects')

url: https://github.com/chrismaltby/gb-studio, score: 0.24864
url: https://github.com/cping/LGame, score: 0.14312
url: https://github.com/watabou/pixel-dungeon, score: 0.13861
url: https://github.com/stride3d/stride, score: 0.13423
url: https://github.com/FlaxEngine/FlaxEngine, score: 0.13415
url: https://github.com/cocos2d/cocos2d-x, score: 0.13239
url: https://github.com/WillKoehrsen/ai-projects, score: 0.12867
url: https://github.com/CKGrafico/Cordova-Multiplatform-Template, score: 0.12794
url: https://github.com/ppy/osu, score: 0.12577
url: https://github.com/UnityTechnologies/open-project-1, score: 0.11988


In [125]:
model = Tfidf()
model.print_results(query='visual recognition and image segmentation tasks')

url: https://github.com/khanhnamle1994/computer-vision, score: 0.20714
url: https://github.com/yassouali/ML-paper-notes, score: 0.18564
url: https://github.com/hoya012/CVPR-2019-Paper-Statistics, score: 0.17400
url: https://github.com/amusi/awesome-ai-awesomeness, score: 0.15092
url: https://github.com/ArcherFMY/Paper_Reading_List, score: 0.14821
url: https://github.com/bnosac/image, score: 0.14183
url: https://github.com/google-research/scenic, score: 0.13965
url: https://github.com/balavenkatesh3322/CV-pretrained-model, score: 0.13816
url: https://github.com/SnailTyan/deep-learning-papers-translation, score: 0.13588
url: https://github.com/microsoft/computervision-recipes, score: 0.13500


<div dir = "rtl">
<font size=3>
<b>FastText:</b> 
</div>

<div dir = "rtl">
در این روش، در ابتدا مدل را با استفاده از توکن‌های مربوط به README ها ترین میکنیم. همچنین امبدینگ مربوط به هر داک را محاسبه میکنیم. پس از آن کوئری را دریافت کرده و با همان مراحلی که داک‌ها را توکنایز کرده بودیم،‌این کوئری را هم توکنایز و نرمالایز می کنیم. هدف آن است که متن و کوئری در یک فضای کلمات قرار گیرند. در گام بعدی بردار امبدینگ مربوط به کوئری را هم حساب میکنیم. در انتها برای آنکه مشخص شود فاصله بین کدام دو بردار امبدینگ (کوئری و داک ها) از همه کمتر است،‌ فاصله کسینوسی را حساب میکنیم. هرچقدر این مقدار کمتر باشد، دو بردار نزدیکتر بوده و در نتیجه داک در رتبه بالاتری است. البته قابل توجه است که مدل های آماده pretrained برا ترین کردن اولیه بهتر از ترین کردن با دیتا به صورت unsupervised است. اما چون کوئری نیز از همان فضای کلمات متن ها آمده است، در نتیجه مشکل زیادی پیش نمی آید. 
مقدار mrr عدد 0.81 است.

</div>

In [127]:
a = ft_model.search_query('deep learning tutorial')
a

[('https://github.com/amusi/Deep-Learning-Interview-Book', 0.9077829122543335),
 ('https://github.com/lexfridman/mit-deep-learning', 0.8877608776092529),
 ('https://github.com/nivu/ai_all_resources', 0.8830912709236145),
 ('https://github.com/goodrahstar/my-awesome-AI-bookmarks',
  0.8822774291038513),
 ('https://github.com/mbadry1/DeepLearning.ai-Summary', 0.879920244216919),
 ('https://github.com/thibo73800/aihub', 0.8784650564193726),
 ('https://github.com/tirthajyoti/Papers-Literature-ML-DL-RL-AI',
  0.877159833908081),
 ('https://github.com/khanhnamle1994/computer-vision', 0.8743525743484497),
 ('https://github.com/fchollet/deep-learning-with-python-notebooks',
  0.8736817836761475),
 ('https://github.com/rasmusbergpalm/DeepLearnToolbox', 0.8701558113098145)]

In [128]:
a = ft_model.search_query('deploy and maintanance')
a

[('https://github.com/Netflix/chaosmonkey', 0.8745149374008179),
 ('https://github.com/spinnaker/spinnaker', 0.8625348806381226),
 ('https://github.com/hashicorp/nomad', 0.8613294959068298),
 ('https://github.com/vercel/vercel', 0.852674126625061),
 ('https://github.com/anaibol/awesome-serverless', 0.8515518307685852),
 ('https://github.com/ToolJet/ToolJet', 0.8509125709533691),
 ('https://github.com/BretFisher/udemy-docker-mastery', 0.8507421612739563),
 ('https://github.com/traefik/traefik', 0.8492861390113831),
 ('https://github.com/aws/aws-cdk', 0.8492258787155151),
 ('https://github.com/donnemartin/awesome-aws', 0.8476577401161194)]

In [129]:
a = ft_model.search_query('ios with swift')
a

[('https://github.com/Aufree/trip-to-iOS', 0.8616037368774414),
 ('https://github.com/jordansinger/messages-multiplatform-swiftui-sample',
  0.8342176079750061),
 ('https://github.com/dkhamsing/open-source-ios-apps', 0.7441614866256714),
 ('https://github.com/draveness/analyze', 0.7218459248542786),
 ('https://github.com/ibireme/YYKit', 0.7206997871398926),
 ('https://github.com/didi/DoKit', 0.6901710629463196),
 ('https://github.com/Tim9Liu9/TimLiu-iOS', 0.6850401163101196),
 ('https://github.com/vsouza/awesome-ios', 0.6763685345649719),
 ('https://github.com/SnapKit/SnapKit', 0.6589732766151428),
 ('https://github.com/h2y/Shadowrocket-ADBlock-Rules', 0.6556643843650818)]

<div dir = "rtl">
<font size=3>
<b>Transformer:</b> 
</div>

<div dir = "rtl">
​​ما از مدل distilbert-base-nli-stsb-mean-tokens استفاده خواهیم کرد که بهترین عملکرد را در تسک های Textual Similarity  در بین نسخه های DistilBERT دارد. علاوه بر این اگرچه این مدل کمی بدتر از BERT است، اما به دلیل داشتن اندازه کوچکتر، بسیار سریعتر است.
 
Faiss  کتابخانه ای برای جستجوی شباهت و خوشه بندی بردارهای دنس است. این شامل الگوریتم‌هایی است که در مجموعه‌هایی از بردارها با هر اندازه حتی آنهایی که در RAM جا نمی‌شوند جستجو می ‌کنند.
 
این یک index ساده که یک جست و جو با فاصله‌ی L2 brute-force که به صورت خطی افزایش پیدا میکند انجام میدهد.
برای ایجاد یک index با بردارهای بدنه readme  این کار را انجام می دهیم:
نوع داده بردارهای readme را به float32 تغییر میدهیم.یک index میسازیم و به اندازه بعد بردارهایی که روی آنها عملیات انجام خواهد شد.ایندکس را به IndexIDMap ارسال میکنیم و آبجکتی که  شیئی که به ما امکان می‌دهد یک لیست دلخواه از شناسه‌ها برای بردارهای ایندکس شده ارائه کنیم.بردارهای readme و نقشه شناسه آنها را به ایندکس اضافه میکنیم که بردارها را به شناسه readme آنها مپ میکنیم.
 
ایندکسی که ما ساختیم یک جستجوی KNN را انجام می دهد و در آخر باید تعداد همسایه هایی که باید برگردانده شوند را نشان میدهیم.
 مقدار mrr برابر با 0.86 است. 
</div>

q1 = "deep learning papers"

```
L2 distance: [205.97442626953125, 210.46890258789062, 213.1353302001953, 213.1353302001953, 214.97361755371094, 218.3271484375, 220.17562866210938, 223.4440460205078, 223.4635772705078, 223.55410766601562]

Readme IDs: [467, 74, 23, 464, 1377, 1082, 270, 508, 172, 1285]
['https://github.com/tflearn/tflearn',
 'https://github.com/floodsung/Deep-Learning-Papers-Reading-Roadmap',
 'https://github.com/kmario23/deep-learning-drizzle',
 'https://github.com/kmario23/deep-learning-drizzle',
 'https://github.com/coqui-ai/TTS',
 'https://github.com/Kulbear/deep-learning-coursera',
 'https://github.com/d2l-ai/d2l-en',
 'https://github.com/baidu-research/DeepBench',
 'https://github.com/ChristosChristofidis/awesome-deep-learning',
 'https://github.com/david-gpu/srez']
```
q2 = "learning resources for computer vision"

```
L2 distance: [138.74569702148438, 169.27291870117188, 173.50767517089844, 176.0240936279297, 178.26512145996094, 179.61874389648438, 182.32862854003906, 183.37486267089844, 185.71066284179688, 185.7895965576172]

Readme IDs: [682, 1330, 420, 1138, 132, 1281, 415, 1232, 1187, 318]
['https://github.com/kenjihata/cs231a-notes',
 'https://github.com/ankonzoid/artificio',
 'https://github.com/llSourcell/Learn_Computer_Vision',
 'https://github.com/khanhnamle1994/computer-vision',
 'https://github.com/microsoft/muzic',
 'https://github.com/dformoso/machine-learning-mindmap',
 'https://github.com/aws-samples/aws-machine-learning-university-accelerated-cv',
 'https://github.com/jrobchin/Computer-Vision-Basics-with-Python-Keras-and-OpenCV',
 'https://github.com/dusty-nv/jetson-inference',
 'https://github.com/chainer/chainercv']
 ```
q3 = "infrastructure based on the asp.net core"

```
L2 distance: [202.02178955078125, 202.56175231933594, 203.72470092773438, 205.60671997070312, 212.73658752441406, 213.1620635986328, 216.88650512695312, 219.00022888183594, 220.93382263183594, 221.25343322753906]

Readme IDs: [475, 818, 772, 526, 251, 711, 1347, 1245, 88, 873]
['https://github.com/thingsboard/thingsboard',
 'https://github.com/bridgecrewio/checkov',
 'https://github.com/abpframework/abp',
 'https://github.com/aspnetboilerplate/aspnetboilerplate',
 'https://github.com/ReactiveX/RxSwift',
 'https://github.com/alibaba/x-deeplearning',
 'https://github.com/graphql/dataloader',
 'https://github.com/Jackpopc/aiLearnNotes',
 'https://github.com/hashicorp/terraform',
 'https://github.com/dotnet-architecture/eShopOnWeb']
```
q4 = "nlp chat bots"

```
L2 distance: [197.46856689453125, 198.1324005126953, 202.7813720703125, 208.62408447265625, 208.9859619140625, 211.4918670654297, 212.06109619140625, 213.52980041503906, 222.63330078125, 222.7926483154297]

Readme IDs: [946, 826, 608, 432, 860, 950, 1185, 1047, 145, 1382]
['https://github.com/BotLibre/BotLibre',
 'https://github.com/LeiWang1999/ZYNQ-NVDLA',
 'https://github.com/DevDungeon/Cathy',
 'https://github.com/chrisalbon/mlai',
 'https://github.com/RubensZimbres/Repo-2016',
 'https://github.com/yanshengjia/artificial-intelligence',
 'https://github.com/deepmipt/DeepPavlov',
 'https://github.com/andelf/PyAIML',
 'https://github.com/getsentry/sentry',
 'https://github.com/aidlearning/AidLearning-FrameWork']
 ```
q7 = "password manager"

 ```
L2 distance: [155.17620849609375, 194.96591186523438, 251.89572143554688, 255.93499755859375, 262.82598876953125, 263.5931701660156, 264.57269287109375, 267.79254150390625, 268.3603210449219, 271.2806701660156]

Readme IDs: [355, 1153, 1249, 1385, 570, 1191, 840, 402, 577, 565]
['https://github.com/marcwebbie/passpie',
 'https://github.com/keepassxreboot/keepassxc',
 'https://github.com/keycloak/keycloak',
 'https://github.com/houtianze/bypy',
 'https://github.com/jpillora/cloud-torrent',
 'https://github.com/owncloud/core',
 'https://github.com/kzlekk/HWSensors',
 'https://github.com/Gazler/githug',
 'https://github.com/facebookarchive/RakNet',
 'https://github.com/GoogleCloudPlatform/python-docs-samples']
 ```






<div dir = "rtl">
<font size=3>
<b>گسترش پرسمان</b> 

در این بخش، برای چهار متدی که قبلا پیاده سازی شده بود، گسترش کوئری انجام شده است. 
برای ولین متفاوت از سایر بخش ها است. در بازیابی بولین تعدادی کلمه داریم که با AND
یا
OR
یا 
NOT
از هم جدا شده اند. برای هریک از کلمات، باساتفاده از مدل 
Fasttext
که از قبل داشتیم، کلمات مشابه را پیدا میکنیم. سپس هر کوئری را تبدیل به چندین کوئری میکنیم به صورتی که به طور رندوم، کلمات را با معادل های آن ها در کوئری جایگزین میکنیم. 

سپس تمام کوئری ها را بازیابی کرده و خروجی نهایی حاصل احتماع آن ها است. 

برای سه مدل دیگر، یک نوع rocchio
پیاده سازی شده است. به این صورت که بردار کوئری با استفاده از بردار داکهای مرتبط و غیرمرتبط آپدیت میشود. در بخش زیر کد مربوط به این بخش پیاده سازی شده است. توجه داریم که برای تمام سه الگوریتم تنها یک پیاده سازی کفایت دارد و در هریک بسته به نوع امبدینگ، ورودی ها متفاوت خواهند بود. باتوجه به اکسل مربوط به MRR
بعد از بهبود کوئری، خروجی ها به شکل معناداری مرتبط تر شده اند. 
</div>

## Rocchio Algorithm 

In [64]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
class Rocchio:

    def __init__(self, alpha = 1, beta = 0.75, gamma = 0.15, iters = 2, rel_count = 7, nrel_count = 3):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.iters = iters
        self.rel_count = rel_count
        self.nrel_count = nrel_count

    def expand_query(self, sorted_docs, query, urls = False):
        '''
        sorted_docs are the result of retrival
        query is a vectorized
        output is a numpy array 
        '''
        
        sorted_docs = np.array(sorted_docs)
        query = np.array(query)

        cos_sim = cosine_similarity(np.array([query]), sorted_docs)
        rankings = np.flip(cos_sim.argsort(axis=1), axis=1)
        for _ in range(self.iters):
            rel_vecs = sorted_docs[rankings[:, :self.rel_count]].mean(axis=1)
            nrel_vecs = sorted_docs[rankings[:, -self.nrel_count:]].mean(axis=1)
            query = self.alpha * query + self.beta * rel_vecs - self.gamma * nrel_vecs
            cos_sim = cosine_similarity(query, sorted_docs)
            rankings = np.flip(cos_sim.argsort(axis=1), axis=1)
        if type(urls) ==bool:
            return sorted_docs[rankings[:, :]], cos_sim[rankings[:,:]]
        else:
            urls = np.array(urls)
            return sorted_docs[rankings[:, :]], urls[rankings[:,:]], cos_sim[0][rankings[:,:]]

            
